In [199]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression

In [200]:
X,Y,coef= make_regression(n_samples=1000,n_features=16,n_targets=1,noise=1, coef=True)

In [201]:
Y= Y.reshape(-1,1)

In [202]:
X.shape,Y.shape,coef.shape

((1000, 16), (1000, 1), (16,))

# 机器学习方法

In [203]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [204]:
reg = LinearRegression()
reg.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [205]:
mean_squared_error(reg.predict(X),Y)

1.066232359438965

# 用来做随机batch的模块

In [212]:
class DiabetesDataset(Dataset):
    def __init__(self, x_, y_):
        self.len = x_.shape[0]
        self.x_data = torch.from_numpy(x_)
        # 这部很关键， 特征类型要求float类型
        self.x_data = self.x_data.float()

        self.y_data = torch.from_numpy(y_)
        self.y_data = self.y_data.float()

    def __getitem__(self, item):
        return self.x_data[item], self.y_data[item]

    def __len__(self):
        return self.len

In [213]:
dataset = DiabetesDataset(X, Y)
train_loader = DataLoader(dataset=dataset, batch_size=100, shuffle=True)

# 网络结构

In [214]:
#网络结构
class NeuralNet(nn.Module):
    def __init__(self, input_size_, hidden_size_, num_classes_):
        super().__init__()
        self.fc1 = nn.Linear(input_size_, hidden_size_) 
        self.fc2 = nn.Linear(hidden_size_, num_classes_)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        return out

# 建立模型训练

In [215]:
model = NeuralNet(16, 8, 1)
#损失函数
criterion = nn.MSELoss(reduction='mean')
#优化算法
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [217]:
total_step = len(train_loader)
num_epochs = 400
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        x = x.reshape(-1, 16)
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #输出中间信息
        if i % 50 == 0 and epoch%20==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/400], Step [1/10], Loss: 55.3455
Epoch [21/400], Step [1/10], Loss: 31.4187
Epoch [41/400], Step [1/10], Loss: 16.9516
Epoch [61/400], Step [1/10], Loss: 11.1000
Epoch [81/400], Step [1/10], Loss: 10.6394
Epoch [101/400], Step [1/10], Loss: 5.9971
Epoch [121/400], Step [1/10], Loss: 3.8124
Epoch [141/400], Step [1/10], Loss: 2.6822
Epoch [161/400], Step [1/10], Loss: 2.0409
Epoch [181/400], Step [1/10], Loss: 1.4825
Epoch [201/400], Step [1/10], Loss: 1.9022
Epoch [221/400], Step [1/10], Loss: 1.6594
Epoch [241/400], Step [1/10], Loss: 1.5809
Epoch [261/400], Step [1/10], Loss: 1.0925
Epoch [281/400], Step [1/10], Loss: 0.8894
Epoch [301/400], Step [1/10], Loss: 1.1790
Epoch [321/400], Step [1/10], Loss: 0.9976
Epoch [341/400], Step [1/10], Loss: 1.0343
Epoch [361/400], Step [1/10], Loss: 0.9275
Epoch [381/400], Step [1/10], Loss: 0.8471


# 保存模型与运行模型

In [218]:
# 保存模型
# torch.save(model, 'model.pkl')

In [219]:
# 加载模型
#model = torch.load('.\model.pkl')
#print(model)
#print(model.parameters())

In [220]:
#预测
#predicted = model(torch.from_numpy(x_data).float())